In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests 
from bs4 import BeautifulSoup
import json
from pprint import pprint
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image
from io import BytesIO
import highlight_text
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
from mplsoccer import Pitch, VerticalPitch
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image
from io import BytesIO
from mplsoccer import PyPizza, add_image, FontManager
from matplotlib.patches import Rectangle
import matplotlib.font_manager as font_manager
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image
from io import BytesIO
from mplsoccer import PyPizza, add_image, FontManager
#import soccerdata as sd
import re
from unidecode import unidecode
from scipy.spatial import ConvexHull
from matplotlib.colors import LinearSegmentedColormap
from highlight_text import ax_text, fig_text


from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import main
import visuals
import seaborn as sns
import requests
import utils
import os
%load_ext autoreload
%autoreload 2

pd.set_option("display.max_rows", None, "display.max_columns", None)
background = "black"
text_color = 'black'
primary = '#C8102E'
secondary = 'lightblue'
mpl.rcParams['xtick.color'] = text_color
mpl.rcParams['ytick.color'] = text_color

## Shot Data - Understat

In [10]:
import understatapi
client = understatapi.UnderstatClient()

In [31]:
## EPL , Bundesliga , Serie_A , Ligue_1 , La_Liga
league = "La_Liga"

league_data = client.league(league=league).get_match_data(season="2024")

In [32]:
matches = []

for match in league_data:
    match_data = {
        'id': match['id'],
        'home_team': match['h']['title'],
        'away_team': match['a']['title'],
        'home_goals': match['goals']['h'],
        'away_goals': match['goals']['a'],
        'home_xG': match['xG']['h'],
        'away_xG': match['xG']['a'],
        'datetime': match['datetime']
        #'forecast': match['forecast']
    }
    matches.append(match_data)

# Convert the list of dictionaries into a DataFrame
matches_df = pd.DataFrame(matches)

In [33]:
all_shot_data = []

import datetime

current_timestamp = datetime.datetime.now().timestamp()

for index,row in matches_df.iterrows():
    match_datetime = pd.to_datetime(row['datetime']).timestamp()
    
    if match_datetime <= current_timestamp:
        #print(row['id'])
        try:
            shot_data = client.match(match=row['id']).get_shot_data()
        except Exception as e:
            print(e)
        all_shot_data.append(shot_data)

27099 is not a valid match


In [34]:
# Initialize an empty list to hold individual shot records
compiled_shot_data = []

# Loop through each match in `all_shot_data`
for match in all_shot_data:
    # Get home shots and away shots from the match
    home_shots = match.get('h', [])
    away_shots = match.get('a', [])
    
    # Add a column to indicate if it's a home or away shot, then extend our list
    for shot in home_shots:
        shot['h_a'] = 'h'  # Indicate as home shot
        compiled_shot_data.append(shot)
    
    for shot in away_shots:
        shot['h_a'] = 'a'  # Indicate as away shot
        compiled_shot_data.append(shot)

# Convert the list of shot records to a DataFrame
compiled_shot_df = pd.DataFrame(compiled_shot_data)

In [35]:
compiled_shot_df.to_csv(f'C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/Data/{league}/2024-25/shot_data.csv')

## Event Data - WhoScored

In [2]:
league_urls = main.getLeagueUrls()
# Updated league urls format: {country-league_name: url}
league_urls

{'Africa-CAF-Champions-League': 'https://1xbet.whoscored.com/Regions/248/Tournaments/290/Africa-CAF-Champions-League',
 'Africa-CAF-Super-Cup': 'https://1xbet.whoscored.com/Regions/248/Tournaments/573/Africa-CAF-Super-Cup',
 'Africa-Africa-Cup-of-Nations-Qualification': 'https://1xbet.whoscored.com/Regions/248/Tournaments/747/Africa-Africa-Cup-of-Nations-Qualification',
 'Africa-African-Nations-Championship-Qualification': 'https://1xbet.whoscored.com/Regions/248/Tournaments/754/Africa-African-Nations-Championship-Qualification',
 'Africa-CAF-Champions-League-Qualification': 'https://1xbet.whoscored.com/Regions/248/Tournaments/762/Africa-CAF-Champions-League-Qualification',
 'Africa-CAF-Confederations-Cup': 'https://1xbet.whoscored.com/Regions/248/Tournaments/574/Africa-CAF-Confederations-Cup',
 'Africa-CECAFA-Senior-Challenge-Cup': 'https://1xbet.whoscored.com/Regions/248/Tournaments/505/Africa-CECAFA-Senior-Challenge-Cup',
 'Asia-Gulf-Cup': 'https://1xbet.whoscored.com/Regions/249/To

In [3]:
#https://1xbet.whoscored.com/Regions/101/Tournaments/582/India-Indian-Super-League
match_urls = main.getMatchUrls(comp_urls=league_urls, competition='England-Premier-League', season='2024/2025')
match_urls[:5]

[{'away': 'Fulham',
  'date': 'Friday, Aug 16 2024',
  'home': 'Manchester United',
  'score': '1:0',
  'url': '/Matches/1821049/Live/England-Premier-League-2024-2025-Manchester-United-Fulham'},
 {'away': 'Brighton',
  'date': 'Saturday, Aug 17 2024',
  'home': 'Everton',
  'score': '0:3',
  'url': '/Matches/1821054/Live/England-Premier-League-2024-2025-Everton-Brighton'},
 {'away': 'Southampton',
  'date': 'Saturday, Aug 17 2024',
  'home': 'Newcastle',
  'score': '1:0',
  'url': '/Matches/1821055/Live/England-Premier-League-2024-2025-Newcastle-Southampton'},
 {'away': 'Aston Villa',
  'date': 'Saturday, Aug 17 2024',
  'home': 'West Ham',
  'score': '1:2',
  'url': '/Matches/1821057/Live/England-Premier-League-2024-2025-West-Ham-Aston-Villa'},
 {'away': 'Liverpool',
  'date': 'Saturday, Aug 17 2024',
  'home': 'Ipswich',
  'score': '0:2',
  'url': '/Matches/1821050/Live/England-Premier-League-2024-2025-Ipswich-Liverpool'}]